In [1]:
import os
import json
import pandas as pd
import numpy as np
import openai
from dotenv import load_dotenv
from tqdm.notebook import tqdm

load_dotenv('~/.secrets')

# use open ai to create embeddings
openai.api_key = os.getenv('OPENAI_KEY')

%reload_ext dotenv
%dotenv

Get the transcripts + video urls in one big data object...

In [2]:
def load_transcripts(file_name: str):
    # load text file transcript.txt 
    df = pd.read_csv('../data/transcribed/' + file_name, on_bad_lines='skip')
    return df

def compare_string(title1, title2):
    words1 = title1.split(' ')
    words2 = title2.split(' ')
    # check if first 3 words are the same
    if words1[:3] == words2[:3]:
        return True
    
def get_video_url(file_name: str):
    for video in video_data:
        if compare_string(video['title'], file_name.split('.')[0]):
            return video['url']

transcribed_dir = os.listdir('../data/transcribed')
video_data = json.load(open('../data/videos.json', 'r'))

transcript_data = []
for file in transcribed_dir:
    df = load_transcripts(file)
    video_url = get_video_url(file)

    transcript_data.append({
        'video_url': video_url,
        'file': file,
        'transcript': df,
    })

Okay, now that all the data is in one place, we can embed multiple transcripts in a batched way...

In [47]:
def get_embeddings(line: str):
    return openai.Embedding.create(input=[line], model="text-embedding-ada-002")["data"][0]["embedding"]

# create the embedding for store the emebdding and line
embedding_type = np.dtype([
    ('embedding', np.float64, (1536,)), 
    ('line', str, 10000),
    ('start_time', np.float64),
    ('video_url', str, 1000),
])

# initialize the embeddings array
embeddings = []

# how many (s) to embed in 1 go
DURATION = 10

# iterate through each transcript
for transcript in transcript_data:
    print("Embedding video: " ,transcript['file'])
    video_url = transcript['video_url']
    df = transcript['transcript']

    embed_text = ''
    text_duration = 0
    start_time = 0
    # iterate through each line in the transcript
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        line = row['text']
        # convert time to seconds
        start, end = row['start'] / 1000, row['end'] / 1000

        # if the time diff is less than embed duration, add to embed text
        if text_duration < DURATION:
            embed_text += line + ' '
            text_duration += end - start
            continue
        
        arr = (
            get_embeddings(embed_text),
            embed_text,
            start_time,
            video_url
        )
        embeddings.append(arr)
        # restart from this line
        embed_text = line
        start_time = end
        text_duration = 0

embeddings = np.array(embeddings, dtype=embedding_type)
# remove embeddings with sum 0
embeddings = embeddings[~np.all(embeddings['embedding'] == 0, axis=1)]

Embedding video:  Are You Smarter Than Me？ - WAN Show April 21, 2023.wav.csv


  0%|          | 0/2155 [00:00<?, ?it/s]

Embedding video:  You Deserve Better - WAN Show May 26, 2023.wav.csv


  0%|          | 0/3041 [00:00<?, ?it/s]

Embeddings generated, now time to insert!

In [6]:
embeddings = np.load('embeddings.npy', allow_pickle=True)

from sqlalchemy import Integer, String, create_engine, text, JSON, insert, Float
from sqlalchemy.orm import declarative_base, mapped_column, Session
from pgvector.sqlalchemy import Vector

postgres_pwd = os.environ.get('POSTGRES_PWD')
# supabase engine
db_uri = 'postgresql://postgres:{}@db.wiagwfwjtbojsbzmeduv.supabase.co:5432/postgres'.format(postgres_pwd)
engine = create_engine(db_uri)

Base = declarative_base()

class Document(Base):
    """
    A class used to represent a Document
    """

    __tablename__ = 'docs'

    id = mapped_column(Integer, primary_key=True, autoincrement=True)
    embedding = mapped_column(Vector(1536))
    line = mapped_column(String)
    meta = mapped_column(JSON)
    video_url = mapped_column(String)
    timestamp = mapped_column(Float)
    created_at = mapped_column(String, server_default=text('NOW()'))
    

# TODO: what does this line do?
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

documents = [
    dict(
        embedding=embedding['embedding'], 
        line=embedding['line'],
        meta={
            'line': embedding['line'], 
            'start_time': embedding['start_time']
        },
        video_url=embedding['video_url'],
        timestamp=embedding['start_time']
    ) for embedding in embeddings
]

session = Session(engine)
session.execute(insert(Document), documents)
session.commit()

In [51]:
# saving embeddings locally
np.save('embeddings.npy', embeddings)

Also append data about youtube to DB...

In [5]:
from sqlalchemy import Integer, String, create_engine, text, JSON, insert, Float
from sqlalchemy.orm import declarative_base, mapped_column, Session
from pgvector.sqlalchemy import Vector

postgres_pwd = os.environ.get('POSTGRES_PWD')
# supabase engine
db_uri = 'postgresql://postgres:{}@db.wiagwfwjtbojsbzmeduv.supabase.co:5432/postgres'.format(postgres_pwd)
engine = create_engine(db_uri)

Base = declarative_base()

class Videos(Base):
    """
    A class used to represent a Document
    """

    __tablename__ = 'videos'

    id = mapped_column(Integer, primary_key=True, autoincrement=True)
    title = mapped_column(String)
    url = mapped_column(String)
    created_at = mapped_column(String, server_default=text('NOW()'))
    

# TODO: what does this line do?
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

videos = [
    dict(
        title=video['file'].split('.')[0],
        url=video['video_url']
    ) for video in transcript_data
]

session = Session(engine)
session.execute(insert(Videos), videos)
session.commit()